# buscador-da-ditadura
Esse script recebe diversos arquivos .csv, já limpos e bem formatados, e detecta quais deles contém menções à Ditadura Militar.

#### Importa pacotes

In [1]:
import datetime
import os
import pandas as pd
import re
from unidecode import unidecode

#### Lê os dados externos e formata strings

In [2]:
df = pd.read_csv("../data/csvs/discursos_jair_bolsonaro.csv")

In [3]:
df["CLEAN_CONTENT_LOWERCASE"]    = df.CLEAN_CONTENT.str.lower()
df["FULL_CLEAN_QUOTE_LOWERCASE"] = df.FULL_CLEAN_QUOTE.str.lower()

#### Define palavras chave para buscar no discurso
Para encontrar pronunciamentos em que Bolsonaro mencionou a Ditadura Militar, vamos definir uma sério de palavras-chave relacionadas ao tema e checar se elas foram usadas em cada um dos discursos coletados.

Importante notar que vamos passar um *unidecode* nas palavras, para que elas ficam sem caracteres especiais, da mesma maneira que a transcrição dos discursos.

#### Descobre quais palavras-chave foram usadas

In [4]:
# Descobre quais palavras foram encontradas em cada discurso
def find_words(string, keywords):
    
    used_words = [ ]
    
    for word in keywords:
        
        regex = re.compile( "\\b" + unidecode(word) + "\\b" )
        matches = re.findall( regex, string.lower() )
        
        if matches != [ ] :
            matches = set(matches)
            used_words.extend( matches )
            
    return used_words

#### Encapsula as funções acima em métodos df.apply

In [5]:
def apply_find_words(row, keywords):
    
    used_words = find_words(row.FULL_CLEAN_QUOTE, keywords)
    
    if used_words != [ ]:
        mentions_keywords = 1
    else:
        mentions_keywords = 0
        
    return pd.Series({
        "MENTIONS_KEYWORDS" : mentions_keywords,
        "USED_WORDS"        : used_words
    })

#### Executa a operação

In [6]:
def run_script():
    
    df = pd.read_csv("../data/csvs/discursos_jair_bolsonaro.csv")
    
    df = df[ df.SESSION_TYPE != 'COMISSÃO ESPECIAL (FORA DO PLENÁRIO)' ]
    
    keywords = [ 

        # Termos genéricos para falar do proceso histórico
        "31 de março",
        "1964",
        "64",
        "ai-5",
        "anistia",
        "contragolpe",
        "ditadura",
        "golpe",
        "governo militar",
        "regime militar",
        "revolução",

        # Citações aos presidentes militares ou outras lideranças
        "alberto mendes junior",
        "castelo branco",
        "castello branco",
        "costa e silva",
        "mario kozel filho",
        "médici",
        "geisel",
        "figueiredo",
        "sergio fleury",
        "ustra",

        # TO DO:
        # Recuperar lista com nomes dos 377 nomes apontados
        # como responsáveis por violações de Direitos Humanos
        # na Comissão da Verdade

        # Nomes de inimigos do Estado durante a ditadura
        # ou de episódios de resistência ao regime
        "aln",
        "araguaia",
        "colina",
        "guerrilha",
        "lamarca",
        "m3g",
        "marighella",
        "molipo",
        "mr-8",
        "mr-21",
        "mr-26",
        "mrmn",
        "prt",
        # "ran",
        "var-palmares",
        "vpr",

        # Instituições
        "comissão parlamentar memória, verdade e justiça"
        "comissao nacional da verdade",
        "comissao da verdade",
        "doi-codi",
        "dops",
        "sni",

        # Nome de outras pessoas direta ou indiretamente envolvidas com o regime
        "elbrick", # Charles Burke, embaixador americano
        "herzog",
        "vladmir herzog",
        "vladimir herzog",
        "joao goulart",
        "goulart",
        "jango",

    ]
    
    found_words = df.apply(apply_find_words, args = (keywords,), axis = 1)
    df = df.join(found_words)
    
    now = str( datetime.datetime.now() )
    fp = f"../data/csvs/discursos_jair_bolsonaro_classificados_{now}.csv"
    df.to_csv(fp, index = False)
    
    return df

In [7]:
df = run_script()